In [1]:
pip install requests


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import requests
import json
import pandas as pd

class REEAPIClient:
    BASE_URL = "https://apidatos.ree.es/es/datos"  # The base URL for the API

    def __init__(self):
        pass

    def fetch_data(self, endpoint, params=None):
        """
        Fetch data from the REE API.
        
        :param endpoint: The endpoint of the API (e.g., '/balance-electrico').
        :param params: Dictionary of query parameters (e.g., {"start_date": "2024-01-01", "end_date": "2025-01-19"}).
        :return: JSON response or error message.
        """
        url = f"{self.BASE_URL}/{type}/{endpoint}"
        try:
            response = requests.get(url, params=params)
            response.raise_for_status()  # Raise an HTTPError for bad responses
            return response.json()  # Parse JSON data
        except requests.exceptions.RequestException as e:
            print(f"Error fetching data from REE API: {e}")
            return None

# Example usage
if __name__ == "__main__":
    # Create an instance of the API client
    client = REEAPIClient()
    type = "balance"
    # Example: Fetch data from a specific endpoint
    endpoint = "balance-electrico"  # Replace with the actual endpoint
    params = {
        "start_date": "2024-01-01T00:00",
        "end_date": "2024-12-31T23:59",
        "time_trunc": "day"
    }
    data = client.fetch_data(endpoint, params)

    if data:
        print("API Response:", data)
    else:
        print("Failed to fetch data.")


# List to store extracted rows
rows = []

# Function to recursively extract data from the JSON structure
def extract_values(data, energy_type, group_id):
    for content_entry in data.get('content', []):  # Navigate to the "content" key
        content_type = content_entry.get('type')
        content_id = content_entry.get('id')
        content_group_id = group_id
        for value_entry in content_entry['attributes'].get('values', []):
            rows.append({
                "Type": content_type,
                "ID": content_id,
                "GroupID": content_group_id,
                "EnergyType": energy_type,
                "Value": value_entry.get('value'),
                "Percentage": value_entry.get('percentage'),
                "Datetime": value_entry.get('datetime'),
            })

# Extract from "included"
for entry in data.get('included', []):
    energy_type = entry.get('type')
    group_id = entry.get('id')
    attributes = entry.get('attributes', {})
    extract_values(attributes, energy_type, group_id)

# Convert extracted rows to a DataFrame
df = pd.DataFrame(rows)

print(df)
print(df.head())




API Response: {'data': {'type': 'Balance de energía eléctrica', 'id': 'bal1', 'attributes': {'title': 'Balance de energía eléctrica', 'last-update': '2025-01-17T10:28:07.000+01:00', 'description': 'Balance eléctrico: asignación de unidades de producción según combustible principal. La producción neta de las instalaciones no renovables e hidráulicas UGH tiene descontados los consumos propios de las instalaciones. En dichos tipos de producción una generación negativa indica que la electricidad horaria consumida para los usos de las plantas excede la producción horaria de las instalaciones.'}, 'meta': {'cache-control': {'cache': 'HIT', 'expireAt': '2025-02-19T18:35:31'}}}, 'included': [{'type': 'Renovable', 'id': 'Renovable', 'attributes': {'title': 'Renovable', 'last-update': '2025-01-17T10:28:03.000+01:00', 'description': None, 'magnitude': None, 'content': [{'type': 'Hidráulica', 'id': '10288', 'groupId': 'Renovable', 'attributes': {'title': 'Hidráulica', 'description': '10288', 'color